# low-rank Krylov Subspace Approximation (KSA) for density matrix SCF optimization

In [8]:
import torch
from seqm.seqm_functions.constants import Constants
from seqm.Molecule import Molecule
from seqm.ElectronicStructure import Electronic_Structure
from seqm.seqm_functions.read_xyz import read_xyz

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [9]:
%%time
DTYPE = torch.float64
torch.set_default_dtype(DTYPE)
files = [
        'C380.xyz',
        ]

species, coordinates = read_xyz(files)
species = torch.as_tensor(species,dtype=torch.int64, device=device)[:]
coordinates = torch.tensor(coordinates, device=device, dtype=DTYPE)[:]

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))

# parapeters dict for KSA-XL-BOMD needs to be passed to 'scf_converger'
xl_bomd_params={'k':6, 'max_rank':3, 'err_threshold':0.0, 'T_el':1500}

seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [3,xl_bomd_params], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                                         # [3, xl_bomd_params], low-rank KSA scf optimization
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

molecules = Molecule(const, seqm_parameters, coordinates, species).to(device)

### Create electronic structure driver:
esdriver = Electronic_Structure(seqm_parameters).to(device)

### Run esdriver on molecules:
esdriver(molecules)


CPU times: user 4min 2s, sys: 3.75 s, total: 4min 5s
Wall time: 18.1 s


In [10]:
print(' Total Energy (eV):\n', molecules.Etot)
print('\n Electronic Energy (eV): ', molecules.Eelec)
print('\n Nuclear Energy (eV):\n', molecules.Enuc)
print('\n Heat of Formation (ev):\n', molecules.Hf)
print('\n HOMO-LUMO gap (eV):\n', molecules.e_gap)
print('\n Forces (eV/A):\n', molecules.force)

 Total Energy (eV):
 tensor([-48587.7154], device='cuda:0')

 Electronic Energy (eV):  tensor([-1787645.6602], device='cuda:0')

 Nuclear Energy (eV):
 tensor([1739057.9448], device='cuda:0')

 Heat of Formation (ev):
 tensor([138.2180], device='cuda:0')

 HOMO-LUMO gap (eV):
 tensor([[2.7302]], device='cuda:0')

 Forces (eV/A):
 tensor([[[-1.1824e-04,  2.9804e-04, -5.9886e-04],
         [-6.8664e-04,  2.0150e-04, -3.7463e-04],
         [ 1.2534e-04,  3.0341e-04, -4.4419e-04],
         ...,
         [-3.7704e-04, -8.6599e-04, -9.3367e-04],
         [ 6.4111e-04, -3.8037e-04,  5.6029e-04],
         [-1.9480e-03,  1.0827e-03, -5.3333e-05]]], device='cuda:0')
